# Import các thư viện cần thiết

In [1]:
import os
from tqdm import tqdm
from typing import List
import pandas as pd

from langdetect import detect
from langdetect import DetectorFactory

# Display all columns
pd.set_option('display.max_columns', None)

# 1. Tiền xử lý dữ liệu transcription từ video

Liệt kê đường dẫn đến các file .csv cần thiết

In [2]:
def list_file_types(directory: str, file_extension: str) -> List[str]:
    """ List all files with a specific extension in a directory.

    Args:
        directory (str): Directory path.
        file_extension (str): File extension.

    Returns:
        List[str]: List of file paths.
    """

    file_list: List[str] = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(file_extension):
                file_list.append(os.path.join(root, file))
    return file_list


transcription_files = list_file_types("data/VideoTranscription", ".csv")
transcription_files[:5]

['data/VideoTranscription/Kaggle/0000000000000000_audio_transcription_12800_13000.csv',
 'data/VideoTranscription/Kaggle/0000000000000000_audio_transcription_13000_13200.csv',
 'data/VideoTranscription/Kaggle/0000000000000000_audio_transcription_13200_13400.csv',
 'data/VideoTranscription/Kaggle/0000000000000000_audio_transcription_13400_13600.csv',
 'data/VideoTranscription/Kaggle/0000000000000000_audio_transcription_13600_13800.csv']

Đọc lần lượt các file .csv thành các DataFrame và lưu vào một list. Đồng thời tạo thêm một cột `url` chứa đường dẫn đến TikTok video tương ứng với mỗi transcription.


In [3]:
# Create a function to format TikTok URLs
def format_tiktok_url(author_id: str, video_id: str) -> str:
    """ Format TikTok URL.

    Args:
        author_id (str): ID of the author.
        video_id (str): ID of the video.

    Returns:
        str: TikTok URL.
    """

    return f"https://www.tiktok.com/@{author_id}/video/{video_id}"

In [4]:
df_list: List[pd.DataFrame] = []

for transcription_file in tqdm(transcription_files):
    # Load the CSV file
    df = pd.read_csv(transcription_file)

    # Apply the function to create or update the URL column
    df['url'] = df.apply(lambda row: format_tiktok_url(
        row['author_id'], row['video_id']), axis=1)

    # Select columns order
    df[['author_id', 'video_id', 'video_transcription', 'url']]

    # Append the DataFrame to the list
    df_list.append(df)

100%|██████████| 75/75 [00:02<00:00, 26.11it/s]


Merge các DataFrame trong list thành một DataFrame duy nhất.

In [5]:
# Concatenate all DataFrames in the list
transcription_df = pd.concat(df_list, ignore_index=True)
transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33501 entries, 0 to 33500
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            33501 non-null  int64 
 1   video_id             33501 non-null  int64 
 2   video_transcription  28929 non-null  object
 3   url                  33501 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.0+ MB


**Nhận xét:**

- Ta thấy có nhiều video không có transcription, ta sẽ loại bỏ các dòng này.

Loại bỏ các dòng bị thiếu dữ liệu.

In [6]:
# Remove rows with missing values
transcription_df = transcription_df.dropna(
    axis='index', how='any'
)

# Reset index after removing rows
transcription_df = transcription_df.reset_index(drop=True)

transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28929 entries, 0 to 28928
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            28929 non-null  int64 
 1   video_id             28929 non-null  int64 
 2   video_transcription  28929 non-null  object
 3   url                  28929 non-null  object
dtypes: int64(2), object(2)
memory usage: 904.2+ KB


Loại bỏ các dòng có transcription ít hơn 50 từ (word).

In [ ]:
# Remove rows with less than 50 words in the transcription
transcription_df = transcription_df[
    transcription_df['video_transcription'].str.split().apply(len) >= 50
]

# Reset index after removing rows
transcription_df = transcription_df.reset_index(drop=True)

transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23825 entries, 0 to 23824
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            23825 non-null  int64 
 1   video_id             23825 non-null  int64 
 2   video_transcription  23825 non-null  object
 3   url                  23825 non-null  object
dtypes: int64(2), object(2)
memory usage: 744.7+ KB


Loại bỏ các dòng có transcription không phải tiếng Việt.

In [8]:
# Create a new column for the detected language
transcription_df["language"] = ["error"] * len(transcription_df)

# Detect the language of the transcription
for index in tqdm(range(len(transcription_df))):
    try:
        DetectorFactory.seed = 0
        transcription_df.loc[index, "language"] = detect(
            transcription_df.loc[index, "video_transcription"])
    except:
        transcription_df.loc[index, "language"] = "error"

100%|██████████| 23825/23825 [01:33<00:00, 254.93it/s]


Số lượng transcription bằng tiếng Việt còn lại sau khi loại bỏ các dòng không hợp lệ.

In [9]:
sum(transcription_df['language'] == 'vi')

23765

In [10]:
# Remove rows with non-Vietnamese transcriptions
transcription_df = transcription_df[
    transcription_df['language'] == 'vi'
]

# Reset index after removing rows
transcription_df = transcription_df.reset_index(drop=True)

transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23765 entries, 0 to 23764
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            23765 non-null  int64 
 1   video_id             23765 non-null  int64 
 2   video_transcription  23765 non-null  object
 3   url                  23765 non-null  object
 4   language             23765 non-null  object
dtypes: int64(2), object(3)
memory usage: 928.4+ KB


In [11]:
transcription_df.sample(n=5)

,author_id,video_id,video_transcription,url,language
12927,83383179737,7312423049859566850,ly chè có giá 90.000đ đắt nhất Hà Nội còn đắt ...,https://www.tiktok.com/@83383179737/video/7312...,vi
13332,6557645879024320514,7383670988795399431,cuối tuần ở Hà Nội ăn chơi gì thủ đô vào hè nh...,https://www.tiktok.com/@6557645879024320514/vi...,vi
15475,6963937987944432642,7452239360126192914,gửi cho mọi người một món ăn đặc sản nhất định...,https://www.tiktok.com/@6963937987944432642/vi...,vi
5593,6566782197955592194,7377653143015804167,học tiếng Anh Hội chợ bán 600.000 đi ăn buffet...,https://www.tiktok.com/@6566782197955592194/vi...,vi
14396,6859343544352687105,7387777072032124167,Tối nay Vĩnh lục tủ đồ khô thì phát hiện Còn m...,https://www.tiktok.com/@6859343544352687105/vi...,vi


Chuyển kiểu dữ liệu của tất cả các cột thành kiểu `str`

In [12]:
# Convert data type of all columns to string
transcription_df = transcription_df.astype(str)
transcription_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23765 entries, 0 to 23764
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   author_id            23765 non-null  object
 1   video_id             23765 non-null  object
 2   video_transcription  23765 non-null  object
 3   url                  23765 non-null  object
 4   language             23765 non-null  object
dtypes: object(5)
memory usage: 928.4+ KB


Loại bỏ các dòng bị trùng lặp. Dựa trên cột `video_id` để xác định dòng nào là trùng lặp. Với các dòng trùng lặp, ta chỉ giữ lại dòng đầu tiên.

In [17]:
# Display number of rows before deduplication
print(f"Before: {len(transcription_df)}")

# Remove duplicates based on video_id column, keeping the first occurrence
transcription_df = transcription_df.drop_duplicates(
    subset=["video_id"], keep='first'
)

# Reset index after deduplication
transcription_df = transcription_df.reset_index(drop=True)

# Display number of rows after deduplication
print(f"After: {len(transcription_df)}")

Before: 23099
After: 23099


# 2. Tiền xử lý dữ liệu thống kê từ video

Đọc dữ liệu thống kê video thành DataFrame.

In [18]:
video_info_df = pd.read_csv("data/interim/video_info.csv", low_memory=False)
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32603 entries, 0 to 32602
Columns: 136 entries, AIGCDescription to audio_to_text
dtypes: bool(20), float64(37), int64(36), object(43)
memory usage: 29.5+ MB


Loại bỏ các cột có tỷ lệ thiếu dữ liệu lớn hơn 50%

In [19]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display first 5 rows
missing_ratio.head()

AIGCDescription      1.0
music.coverThumb     1.0
music.coverMedium    1.0
music.coverLarge     1.0
audio_to_text        1.0
dtype: float64

In [20]:
# Remove columns with missing ratios greater than 0.50
video_info_df = video_info_df.dropna(
    axis='columns', thresh=0.50 * len(video_info_df)
)

In [23]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display the missing ratio
for column, ratio in missing_ratio[:5].items():
    print(f"{column:50}:{ratio:8.2%}")

duetEnabled                                       :  20.29%
stitchEnabled                                     :  19.87%
video.claInfo.originalLanguageInfo.language       :  18.91%
video.claInfo.originalLanguageInfo.canTranslateRealTimeNoCheck:  18.91%
video.claInfo.originalLanguageInfo.languageID     :  18.91%


In [24]:
# video_info_df.info()

Vì các cột có tên bắt đầu với `stats.*` chứa cùng thông tin với các cột có tên bắt đầu với `statsV2.*`, nhưng không có thông tin về `repostCount` như `statsV2.*`. Nên ta sẽ loại bỏ các cột có tên bắt đầu với `stats.*`.


In [25]:
# Remove columns starting with "stats."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
            if not column.startswith("stats.")]
]

# video_info_df.info()

Xóa các cột bắt đầu với `video.claInfo.originalLanguageInfo.*` vì chúng chứa thông tin không cần thiết.


In [26]:
# Remove columns starting with "video.claInfo.originalLanguageInfo."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
            if not column.startswith("video.claInfo.originalLanguageInfo.")]
]

# video_info_df.info()

Tạo một cột chứa danh sách các `hashtag` được trích xuất từ mô tả video. Và tính số lượng hashtag trong mỗi video.

In [27]:
# Replace missing values in "desc" column with an empty string
video_info_df["desc"] = video_info_df["desc"].fillna("")
video_info_df["desc"] = video_info_df["desc"].astype(str)
video_info_df["desc"] = video_info_df["desc"].str.strip()

# Create a new column for the hashtags
# and the number of hashtags in each video
video_info_df["hashtags"] = [""] * len(video_info_df)
video_info_df["num_hashtags"] = [0] * len(video_info_df)

# Extract hashtags from the "desc" column
# and Get the number of hashtags in each video
for index in tqdm(range(len(video_info_df))):
    # Get the description of the video
    description = video_info_df.loc[index, "desc"].strip().lower()

    if description:
        # Remove emojis
        description = description.encode('ascii', 'ignore').decode('ascii')

        # Add a space before all "#" characters
        description = description.replace("#", " #")

        # Find all strings starting with "#" and followed by a word
        hashtags = [word[1:] for word in description.split()
                    if word.startswith("#")]

        # Extract hashtags from the description
        video_info_df.loc[index, "hashtags"] = ",".join(hashtags).strip()

        # Get the number of hashtags
        video_info_df.loc[index, "num_hashtags"] = len(hashtags)
    else:
        video_info_df.loc[index, "hashtags"] = ""
        video_info_df.loc[index, "num_hashtags"] = 0

100%|██████████| 32603/32603 [00:07<00:00, 4202.30it/s]


In [35]:
video_info_df[["hashtags", "num_hashtags"]].sample(n=5)

,hashtags,num_hashtags
3118,"angithuongoi,mukbang,xuhuongtiktok,ancungtikto...",5
2591,"ancungdaune,reviewanngon,ancungtiktok,octrungm...",5
30084,"ancungtiktok,learnontiktok,eatcleanhong",3
22194,"nhanxphanh,ancungtiktok,nuoctuongchinsu,thomng...",5
32325,"ngayquockhanh,happyvietnam,khatvonghungcuong,s...",6


# 3. Nối dữ liệu transcription và dữ liệu thống kê

Merge hai DataFrame transcription và DataFrame thống kê thành một DataFrame duy nhất. Sử dụng videoId làm khóa ngoại.

In [36]:
# Convert data type of video.id to string
video_info_df["video.id"] = video_info_df["video.id"].astype(str)
transcription_df["video_id"] = transcription_df["video_id"].astype(str)

# Merge the DataFrames
merged_df = pd.merge(
    left=video_info_df,
    right=transcription_df,
    how="inner",
    left_on="video.id",
    right_on="video_id"
)

# Reset index after merging
merged_df = merged_df.reset_index(drop=True)

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23099 entries, 0 to 23098
Data columns (total 84 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   CategoryType                     23099 non-null  int64  
 1   author.commentSetting            23099 non-null  int64  
 2   author.downloadSetting           23099 non-null  int64  
 3   author.duetSetting               23099 non-null  int64  
 4   author.ftc                       23099 non-null  bool   
 5   author.id                        23099 non-null  int64  
 6   author.isADVirtual               23099 non-null  bool   
 7   author.isEmbedBanned             23099 non-null  bool   
 8   author.nickname                  23099 non-null  object 
 9   author.openFavorite              23099 non-null  bool   
 10  author.privateAccount            23099 non-null  bool   
 11  author.relation                  23099 non-null  int64  
 12  author.secUid     

Kiểm tra các hàng bị trùng lặp và loại bỏ chúng.

In [38]:
num_duplicates = merged_df.duplicated(subset=["video_id"]).sum()
num_duplicates

np.int64(0)

Loại bỏ các cột có cùng thông tin với các cột khác: `author_id`, `video_id`, v.v.

In [39]:
# Remove redundant columns
redundant_columns = [
    "video_id", "author_id", "language"
]
merged_df = merged_df.drop(columns=redundant_columns, axis=1,
                           errors='ignore')

In [51]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23099 entries, 0 to 23098
Data columns (total 81 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   CategoryType                     23099 non-null  int64  
 1   author.commentSetting            23099 non-null  int64  
 2   author.downloadSetting           23099 non-null  int64  
 3   author.duetSetting               23099 non-null  int64  
 4   author.ftc                       23099 non-null  bool   
 5   author.id                        23099 non-null  int64  
 6   author.isADVirtual               23099 non-null  bool   
 7   author.isEmbedBanned             23099 non-null  bool   
 8   author.nickname                  23099 non-null  object 
 9   author.openFavorite              23099 non-null  bool   
 10  author.privateAccount            23099 non-null  bool   
 11  author.relation                  23099 non-null  int64  
 12  author.secUid     

In [52]:
merged_df.sample(n=2)

,CategoryType,author.commentSetting,author.downloadSetting,author.duetSetting,author.ftc,author.id,author.isADVirtual,author.isEmbedBanned,author.nickname,author.openFavorite,author.privateAccount,author.relation,author.secUid,author.secret,author.signature,author.stitchSetting,author.uniqueId,author.verified,authorStats.diggCount,authorStats.followerCount,authorStats.followingCount,authorStats.friendCount,authorStats.heart,authorStats.heartCount,authorStats.videoCount,collected,createTime,desc,digged,diversificationId,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,item_control.can_repost,music.authorName,music.duration,music.id,music.isCopyrighted,music.original,music.title,officalItem,originalItem,privateItem,secret,shareEnabled,statsV2.collectCount,statsV2.commentCount,statsV2.diggCount,statsV2.playCount,statsV2.repostCount,statsV2.shareCount,stitchDisplay,stitchEnabled,textLanguage,textTranslatable,video.VQScore,video.bitrate,video.claInfo.enableAutoCaption,video.claInfo.hasOriginalAudio,video.codecType,video.definition,video.duration,video.encodedType,video.format,video.height,video.id,video.ratio,video.videoID,video.videoQuality,video.volumeInfo.Loudness,video.volumeInfo.Peak,video.width,collectTime,video.claInfo.captionsType,hashtags,num_hashtags,video_transcription,url
4495,111,0,0,0,False,6927297910691202049,False,False,Khánh Linhh,False,False,0,MS4wLjABAAAADjhrMxi0o1PkYBHZ9Np288C7smyW1mihho...,False,"Hi vọng clip của mình, sẽ giúp ích cho bạn <3\...",0,bepnhalinh,False,756,1100000,127,0,29100000,29100000,858,False,1726750622,"Mời cả nhà cafe dừa mới lạ nha, hông ngon hổng...",False,10042.0,0,True,False,7416337439226121480,False,0,True,Khánh Linhh,57.0,7416337469056043793,False,False,nhạc nền - Khánh Linhh,False,False,False,False,True,957,153,2257,184300,0,208,0,True,vi,True,73.28,1180903.0,True,True,h264,540p,57,normal,mp4,1024,7416337439226121480,540p,v14044g50000crm1svvog65g9i91u5v0,normal,-16.8,0.80353,576,1741176969,1.0,"vatmoingay,tuoimoingay,suatuoi100,vinamilk,anc...",7,Nếu bạn muốn tự tay làm ra được 1 ly nước với ...,https://www.tiktok.com/@6927297910691202049/vi...
13191,0,0,0,0,False,7163686821477729306,False,False,Muoidian,False,False,0,MS4wLjABAAAAI8XcDTgrBv9V3R2Mt4R1hB3ADydjhTq2jc...,False,"Cùng Muoidian khám phá hết ẩm thực, du lịch nh...",0,muoidian,False,6067,256700,282,0,6200000,6200000,1474,False,1734767520,Bánh ống khói Hungary bánh gia truyền ngay Q.8...,False,NaN,0,NaN,False,7450769741779602696,True,0,True,Muoidian,76.0,7450769850660195089,False,False,nhạc nền - Muoidian,False,False,False,False,True,555,68,4626,65100,0,238,0,NaN,vi,True,70.73,2655473.0,True,True,h264,540p,76,normal,mp4,1024,7450769741779602696,540p,v14044g50000ctj770fog65qf7fq9n8g,normal,-5.7,1.00000,576,1741178008,1.0,"muoidian,learnontiktok,ancungtiktok,vtmgr,fly,...",9,bắn thun radio.net thì mọi người nhớ thử đây n...,https://www.tiktok.com/@7163686821477729306/vi...


Lưu DataFrame đã xử lý vào file `merged_data.csv`.

In [53]:
merged_df.to_csv("data/processed/merged_data.csv", index=False)

In [54]:
# merged_df.to_parquet("data/processed/merged_data.parquet", index=False)

# ========================================

In [ ]:
hashtags = video_info_df["hashtags"].apply(
    lambda x: x.split(",")).explode().value_counts()
for index, value in hashtags.items():
    print(f"{index:30}: {value}")

ancungtiktok                  : 17990
learnontiktok                 : 14731
reviewanngon                  : 6593
xuhuong                       : 5612
vtmgr                         : 5355
fyp                           : 5042
                              : 2888
longervideos                  : 2176
foodie                        : 2006
mukbang                       : 1991
anchoivungtau72               : 1928
saigon                        : 1921
vungtau                       : 1919
foodtiktok                    : 1905
dulichtinhbariavungtau        : 1905
food                          : 1764
viral                         : 1684
thanhthoiluottet              : 1561
dulichvungtau                 : 1372
review                        : 1362
mcv                           : 1267
vtvcab                        : 1187
dulich                        : 1043
foodreview                    : 953
trend                         : 947
tiktokfood                    : 947
muoidian                      : 940
tre